<a href="https://colab.research.google.com/github/m-allahkhani/flower-clustering-project/blob/main/projectV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

maryamm allahkhani
shaghayegh shafiee

In [52]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import random
import cv2
from statistics import mode
import math
from google.colab import drive
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score



In [53]:
def load_dataset():
    # image_pickle_file_path = '/content/sample_data/images.pkl'
    # label_pickle_file_path = '/content/sample_data/label.pkl'
    drive.mount('/content/drive')
    # with open(image_pickle_file_path, 'rb') as file:
    #     images = pickle.load(file)

    # with open(label_pickle_file_path, 'rb') as file:
    #     labels = pickle.load(file)
    images = np.load('drive/MyDrive/Colab Notebooks/images.pkl', allow_pickle = True)
    labels = np.load('drive/MyDrive/Colab Notebooks/label.pkl', allow_pickle = True)
    features = []
    # pixel_locations = []
    # hsv_values = []
    for image in images:
        n, m, _ = image.shape
        indices = np.indices((n, m))

        pixel_img = np.column_stack((indices[0].ravel(), indices[1].ravel()))
        # pixel_locations.append(pixel_img)

        hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

        hsv = hsv_image.reshape(n*m, -1)
        # hsv_values.append(hsv)
        feature_image = np.column_stack((hsv,pixel_img))
        features.append(feature_image)
    # return images, labels, pixel_locations, hsv_values , features
    return images ,labels, features

In [54]:
# Classify the datapoints with the Random Forest Classifier
def classify(feature_vectors, labels):
    test_size = 0.2
    X_train, X_test, y_train, y_test = train_test_split(feature_vectors, labels, test_size=test_size, random_state=42)

    clf = RandomForestClassifier(n_estimators=100, random_state=42)

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    return y_test,y_pred

In [55]:

def extract_flower_parts(features):
    numberCluster = 3
    scaler = StandardScaler()
    width = np.max(features[:, 3])#center of the image
    height = np.max(features[:, 4])
    center_x =  width / 2
    center_y = height / 2

    #  horizontal and vertical space each pixel and the center
    horizontal_space = np.abs(features[:, 3] - center_x)
    vertical_space = np.abs(features[:, 4] - center_y)
    horizontal_space_normalized = scaler.fit_transform(horizontal_space.reshape(-1, 1))
    vertical_space_normalized = scaler.fit_transform(vertical_space.reshape(-1, 1))


    num_components = 2    # number of components for GMM
    original_hue_values = features[:, 0].copy()
    gmm = GaussianMixture(n_components=num_components)
    gmm.fit(original_hue_values.reshape(-1, 1))

    # Assign each pixel to the most likely cluster based on its hue value
    cluster_assignments = gmm.predict(original_hue_values.reshape(-1, 1))
    arranged_hue_values = gmm.means_[cluster_assignments]
    normalized_features = scaler.fit_transform(features[:, 1:5])
    normalized_hue_values =scaler.fit_transform(arranged_hue_values.reshape(-1, 1))
    weighted_features = np.empty([len(normalized_features), 7])
    weighted_features[:, 0] = normalized_hue_values.flatten() * 12
    weighted_features[:, 1] = normalized_features[:, 0] * 5
    weighted_features[:, 2] = normalized_features[:, 1] * 8
    weighted_features[:, 3] = normalized_features[:, 2] * 6
    weighted_features[:, 4] = normalized_features[:, 3] * 6
    weighted_features[:, 5] = horizontal_space_normalized.flatten() * 3  #  horizontal space
    weighted_features[:, 6] = vertical_space_normalized.flatten() * 3  #  vertical space

    kmeans = KMeans(n_clusters=numberCluster)
    kmeans.fit(weighted_features)
    cluster_labels = kmeans.predict(weighted_features)
    # cluster_centers = kmeans.cluster_centers_


    labeled_pixels = []
    pixels = []
    for i in range(len(features)):
        pixel = [int(features[i][3]), int(features[i][4]), cluster_labels[i]]
        labeled_pixels.append(pixel)
        pixel2 = [int(features[i][0]), int(features[i][1]), int(features[i][2]), int(features[i][3]),int(features[i][4]), cluster_labels[i]]
        pixels.append(pixel2)

    clusters = [[] for _ in range(numberCluster)]
    for i, pixel in enumerate(pixels):
        cluster_label = pixel[5]
        clusters[cluster_label].append([features[i][0], features[i][1], features[i][2], features[i][3],features[i][4], cluster_label])

    return labeled_pixels,clusters,width,height,len(features)

2 ways to cluster pixels but first is better now .

In [56]:
def show_cluster_image2(pixels, target_label,max_row,max_col): #image_width and height


    # Initialize an empty image with dimensions based on the maximum row and column values
    image = np.zeros((max_row + 1, max_col + 1, 3), dtype=np.uint8)

    # Assign unique colors to each cluster label excluding target label
    label_colors = {}
    for pixel in pixels:
        _, _, label = pixel
        if label not in label_colors:
            if label == target_label:
                label_colors[label] = np.array([255, 0, 0], dtype=np.uint8)
            else:
                while True:
                    color = np.random.randint(0, 256, size=3, dtype=np.uint8)
                    if not np.array_equal(color, [255, 0, 0]): # Avoid red
                        label_colors[label] = color
                        break

    # Fill the image pixels with the corresponding cluster label color
    for pixel in pixels:
        row, col, label = pixel
        color = label_colors[label]
        image[row, col] = color
    # Display the image
    plt.imshow(image)
    plt.axis('off')
    plt.show()


In [59]:
def find_flower_cluster(clusters, rows, cols, pixels_num): #image_width and height
    color_range = 20
    center_weight = 18
    color_weight = 14
    size_weight = 6
    location_weight = 20
    noise_threshold = 0.1
    size_threshold = 0.17
    max_size_thresold = 0.75
    #  center of the image
    center_row = rows // 2
    center_col = cols // 2


    scores = []
    color_differences = []

    for i, cluster in enumerate(clusters):

        target_hue_values = np.array(clusters[i])[:, 0]  # Extract hue values
        # most_hue_target = mode(target_hue_values)
        most_hue_target = int(np.median(target_hue_values))
        if (most_hue_target >= 40 and most_hue_target <= 80):  # Green color range in HSV
            color_green = -1
        else:
            color_green = 0
        for j in range(0,len(clusters)):
            if i != j:
                other_hue_values = np.array(clusters[j])[ :, 0]  # Extract hue values
                # mode_hue_others = mode(other_hue_values)
                mode_hue_others = int(np.median(other_hue_values))
                difference =np.abs(mode_hue_others - most_hue_target)
                color_differences.append(difference)

        color_difference = np.mean(color_differences)
        # Normalize the color difference score
        max_difference = np.max(color_differences)
        min_difference = np.min(color_differences)

        if max_difference != min_difference:
            color_difference = (color_difference - min_difference) / (max_difference - min_difference)
        else:
            color_difference = 0.0
        size = len(cluster) / pixels_num

        cluster_array = np.array(cluster)

        center_distance = np.sqrt((np.mean(cluster_array[:, 3]) - center_row) ** 2 + (np.mean(cluster_array[:, 4]) - center_col) ** 2) #  distance from cluster center to image center

        max_center_distance = np.sqrt(center_row ** 2 + center_col ** 2)  # Normalize the distance
        center_score = 1.0 - (center_distance / max_center_distance)

        location_score = 0.0

        min_row = np.min(cluster_array[:, 3])
        max_row = np.max(cluster_array[:, 3])
        min_col = np.min(cluster_array[:, 4])
        max_col = np.max(cluster_array[:, 4])


        if min_row <= rows // 40  and max_row - min_row >= rows // 3 and min_col <= cols // 40 :
            location_score -= 0.125

        if min_row <= rows // 40  and max_row - min_row >= rows // 3 and  max_col >= cols * 39 // 40:
            location_score -= 0.125


        if max_row >= rows * 39 // 40 and max_row - min_row >= rows // 3 and  max_col >= cols * 39 // 40:
            location_score -= 0.125


        if max_row >= rows * 39 // 40 and max_row - min_row >= rows // 3 and min_col <= cols // 40 :
            location_score -= 0.125
        if min_col <= cols // 40  and max_col - min_col >= cols // 3 and min_row <= rows // 40 :
            location_score -= 0.125
        if min_col <= cols // 40  and max_col - min_col >= cols // 3 and  max_row >= rows * 39 // 40:
            location_score -= 0.125
        if  max_col >= cols * 39 // 40 and max_col - min_col >= cols // 3 and ( max_row >= rows * 39 // 40):
            location_score -= 0.125
        if  max_col >= cols * 39 // 40 and max_col - min_col >= cols // 3 and  max_row >= rows * 39 // 40:
            location_score -= 0.125
        if size <= size_threshold:
            size_score = -1
        else:
            size_score = size

        # total score for the cluster
        total_score = (center_weight * center_score +
                       size_score * size_weight +
                       color_weight * color_difference +
                       location_weight * location_score
                       + color_green * color_range)

        if size <= noise_threshold or size>= max_size_thresold:
            total_score = 0

        scores.append(total_score)

    return np.argmax(scores)

In [60]:
def show_image(image):
    plt.imshow(image)
    plt.axis('off')
    plt.show()

In [61]:
def show_cluster_image(pixels,max_row,max_col): #image_width and height


    # Initialize an empty image with dimensions based on the maximum row and column values
    image = np.zeros((max_row + 1, max_col + 1, 3), dtype=np.uint8)

    # Assign unique colors to each cluster label
    label_colors = {}
    for pixel in pixels:
        _, _, label = pixel
        if label not in label_colors:
            label_colors[label] = np.random.randint(0, 256, size=3, dtype=np.uint8)

    # Fill the image pixels with the corresponding cluster label color
    for pixel in pixels:
        row, col, label = pixel
        color = label_colors[label]
        image[row, col] = color

    # Display the image
    plt.imshow(image)
    plt.axis('off')
    plt.show()

In [62]:
def test_finding_flower(num):
    images2, _, features = load_dataset()
    numn = []
    for j in range(0,num):
        i = random.randrange(0, len(images2), 3)
        while i in numn:
            i = random.randrange(0, len(images2), 3)
        print(i)
        show_image(images2[i])
        pixel_labels2,clusters,width,height,points_num= extract_flower_parts(features[i])
        # print(clusters)
        # print(len(clusters))
        # print("-------------")
        flower_index = find_flower_cluster(clusters,width,height,points_num)
        show_cluster_image2(pixel_labels2,clusters[flower_index][0][5],width,height)

        # flower_index = is_flower_cluster2(clusters,width,height,points_num)
        # show_cluster_image2(pixel_labels2,clusters[flower_index][0][5],width,height)

        numn.append(i)

In [ ]:

test_finding_flower(15)




In [63]:

def calculate_hue_change_coefficients(pixels, image_width, image_height):
    x = np.array([pixel[4] for pixel in pixels])  # Column indices
    y = np.array([pixel[3] for pixel in pixels])  # Row indices
    hue = np.array([pixel[0] for pixel in pixels])  # Hue values

    # distance to center
    distance_to_center = np.sqrt((y - image_height/2)**2 + (x - image_width/2)**2)
    max_distance = np.sqrt((image_height/2)**2 + (image_width/2)**2)
    normalized_distance = distance_to_center / max_distance

    # Left side
    left_indices = x < image_width/2
    a1 = np.sum((hue[left_indices] - np.mean(hue)) * normalized_distance[left_indices]) / np.sum(normalized_distance[left_indices]**2)

    # Right side
    right_indices = x >= image_width/2
    a2 = np.sum((hue[right_indices] - np.mean(hue)) * normalized_distance[right_indices]) / np.sum(normalized_distance[right_indices]**2)

    # Top side
    top_indices = y < image_height/2
    a3 = np.sum((hue[top_indices] - np.mean(hue)) * normalized_distance[top_indices]) / np.sum(normalized_distance[top_indices]**2)

    # Bottom side
    bottom_indices = y >= image_height/2
    a4 = np.sum((hue[bottom_indices] - np.mean(hue)) * normalized_distance[bottom_indices]) / np.sum(normalized_distance[bottom_indices]**2)

    return a1, a2, a3, a4


In [64]:

def calculate_shape(max_width,max_height,flower_pixels): #max_width,max_height  of flower


    # Calculate the area of the flower
    flower_area = max_width * max_height * (flower_pixels / (max_width * max_height))

    # Calculate the uncovered area for each shape
    circle_radius = math.sqrt(flower_area / math.pi)
    circle_area = math.pi * circle_radius ** 2
    circle_uncovered_area = circle_area - flower_area

    triangle_side = math.sqrt((4 * flower_area) / (math.sqrt(3) * 3))
    triangle_area = (math.sqrt(3) / 4) * triangle_side ** 2
    triangle_uncovered_area = triangle_area - flower_area

    rectangle_width = math.sqrt(flower_area / 2)
    rectangle_height = 2 * rectangle_width
    rectangle_area = rectangle_width * rectangle_height
    rectangle_uncovered_area = rectangle_area - flower_area
    # min_uncovered_area = min(circle_uncovered_area, triangle_uncovered_area, rectangle_uncovered_area)

    # if min_uncovered_area == circle_uncovered_area:
    #     min_shape = "circle"
    # elif min_uncovered_area == triangle_uncovered_area:
    #     min_shape = "triangle"
    # else:
    #     min_shape = "rectangle"

    return  circle_area/flower_pixels, triangle_area/flower_pixels, rectangle_area/flower_pixels

In [65]:
def get_features(cluster,points_num, image_width, image_height):
    cluster_array = np.array(cluster)
    min_row = np.min(cluster_array[:, 3])
    max_row = np.max(cluster_array[:, 3])
    min_col = np.min(cluster_array[:, 4])
    max_col = np.max(cluster_array[:, 4])
    width = max_col-min_col
    height = max_row-min_row

    most_hue = mode(cluster_array[:, 0] )
    features_vector = []
    hue_changes1,hue_changes2,hue_changes3,hue_changes4 = calculate_hue_change_coefficients(cluster, image_width, image_height)
    circle,traingle,rectangle = calculate_shape(width,height,points_num) #width,height
    features_vector = [most_hue,hue_changes1,hue_changes2,hue_changes3,hue_changes4,circle,traingle,rectangle]
    return features_vector

In [66]:


def MatrixCal(true_labels,predicted_labels):

# Create the confusion matrix
   confusion_mat = confusion_matrix(true_labels, predicted_labels)

# Create a pandas DataFrame for better visualization
   confusion_df = pd.DataFrame(confusion_mat, index=['True1', 'True2', 'True3', 'True4', 'True5', 'True6', 'True7'],
                             columns=['Pred1', 'Pred2', 'Pred3', 'Pred4', 'Pred5', 'Pred6', 'Pred7'])

   print("Confusion Matrix:")
   print(confusion_df)

# Calculate the precision, recall, f1-score, and accuracy
   precision = precision_score(true_labels, predicted_labels, average='weighted')
   recall = recall_score(true_labels, predicted_labels, average='weighted')
   f1 = f1_score(true_labels, predicted_labels, average='weighted')
   accuracy = accuracy_score(true_labels, predicted_labels)

   print("\nPrecision:", precision)
   print("Recall:", recall)
   print("F1-score:", f1)
   print("Accuracy:", accuracy)


In [67]:


def get_accuracy2(num):
    images, labels, features = load_dataset()

    # random_indices = random.sample(range(len(images2)), num)
    # images = [images2[i] for i in random_indices]
    # labels = [labels2[i] for i in random_indices]
    # features = [features2[i] for i in random_indices]

    features_vector = []
    for i in range(0,len(images)):
        # show_image(images[i])
        pixel_labels2,clusters,image_width, image_height ,points_num= extract_flower_parts(features[i])
        flower_index = find_flower_cluster(clusters,image_width, image_height,points_num)
        # show_cluster_image2(pixel_labels2,clusters[flower_index][0][5],image_width, image_height)
        v = get_features(clusters[flower_index],points_num,image_width, image_height)

        # Convert features_vector to a numpy array
        features_vector_arr = np.array(features_vector)

        # Replace NaN values with integers based on the other lists in features_vector
        for j in range(len(v)):
            if np.isnan(v[j]):
                v[j] = int(np.nanmean(features_vector_arr[:, j]))  # Replace NaN with the mean of the non-NaN values for that feature

        features_vector.append(v)

    features_vector = np.array(features_vector)
    scaler = StandardScaler()
    normalized_features = scaler.fit_transform(features_vector[:, :])

    weighted_features = np.empty([len(normalized_features), 8])
    weighted_features[:, 0] = normalized_features[:, 0] * 3
    weighted_features[:, 1] = normalized_features[:, 1] * 2
    weighted_features[:, 2] = normalized_features[:, 2] * 2
    weighted_features[:, 3] = normalized_features[:, 3] * 2
    weighted_features[:, 4] = normalized_features[:, 4] * 2
    weighted_features[:, 5] = normalized_features[:, 5] * 1
    weighted_features[:, 6] = normalized_features[:, 6] * 1
    weighted_features[:, 7] = normalized_features[:, 7] * 1

    y_test,y_pred =classify(weighted_features, labels)
    MatrixCal(y_test,y_pred)

In [68]:
get_accuracy2(100)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

Accuracy: 56.25%
Confusion Matrix:
       Pred1  Pred2  Pred3  Pred4  Pred5  Pred6  Pred7
True1     13      0      0      4      2      0      0
True2      0     12      1      3      1      3      1
True3      0      5      8      1      0      2      0
True4      1      0      0     10      0      2      0
True5      0      1      1      1      9      4      0
True6      0      2      3      0      0      3      1
True7      0      3      0      5      0      2      8

Precision: 0.6424080303107865
Recall: 0.5625
F1-score: 0.5804489663356659
Accuracy: 0.5625
